In [6]:
from hipt_4k import HIPT_4K
from hipt_model_utils import get_vit256, get_vit4k, eval_transforms
from hipt_heatmap_utils import *
import torch

In [7]:
light_jet = cmap_map(lambda x: x/2 + 0.5, matplotlib.cm.jet)

pretrained_weights256 = '/home/siddhesh/Work/Projects/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth'
pretrained_weights4k = '/home/siddhesh/Work/Projects/HIPT/HIPT_4K/Checkpoints/vit4k_xs_dino.pth'
device256 = torch.device('cpu')
device4k = torch.device('cpu')

### ViT_256 + ViT_4K loaded independently (used for Attention Heatmaps)
model256 = get_vit256(pretrained_weights=pretrained_weights256, device=device256)
model4k = get_vit4k(pretrained_weights=pretrained_weights4k, device=device4k)

### ViT_256 + ViT_4K loaded into HIPT_4K API
model = HIPT_4K(pretrained_weights256, pretrained_weights4k, device256, device4k)
model.eval()

Take key teacher in provided checkpoint dict
Pretrained weights found at /home/siddhesh/Work/Projects/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
# of Patches: 196
Take key teacher in provided checkpoint dict
Pretrained weights found at /home/siddhesh/Work/Projects/HIPT/HIPT_4K/Checkpoints/vit4k_xs_dino.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
Take key teacher in provided checkpoint dict
Pretrained weights found at /home/siddhesh/Work/Projects/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth and loaded with msg: _IncompatibleKeys(

HIPT_4K(
  (model256): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate=none)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): L

In [19]:
import cv2
import numpy as np

def process_image(image_path):
    # Step 1: Read the image
    img = cv2.imread(image_path)
    if img is None:
        return "Invalid image path"

    h, w, _ = img.shape

    # Step 2: Check Dimensions
    if h < 4096 or w < 4096:
        # Step 3a: Padding
        pad_h = 4096 - h if h < 4096 else 0
        pad_w = 4096 - w if w < 4096 else 0
        padded_img = cv2.copyMakeBorder(img, 0, pad_h, 0, pad_w, cv2.BORDER_REFLECT, value=[0, 0, 0])
        return padded_img

    else:
        # Step 3b: Slicing into 4096x4096 patches
        patches = []
        for i in range(0, h - 4096 + 1, 4096):
            for j in range(0, w - 4096 + 1, 4096):
                patch = img[i:i + 4096, j:j + 4096]
                patches.append(patch)
        
        patches = np.array(patches)  # Convert list to numpy array (N x 4096 x 4096 x 3)
        return patches

# Test the function
image_path = "/media/siddhesh/D1-k/Kaggle-Ovarian/train_images/970.png"
result = process_image(image_path)

In [21]:
# import matplotlib.pyplot as plt

# plt.imshow(result)
result.shape

(28, 4096, 4096, 3)

In [22]:
# For every patch, iterate over all the tiles and get the attention maps

for i in range(result.shape[0]):
    

ValueError: pic should be 2/3 dimensional. Got 4 dimensions.

tensor([[[[0.7333, 0.7412, 0.7333,  ..., 0.6863, 0.6784, 0.7412],
          [0.7098, 0.7412, 0.7490,  ..., 0.6941, 0.6941, 0.6863],
          [0.6941, 0.7333, 0.7176,  ..., 0.7020, 0.7176, 0.6392],
          ...,
          [0.7098, 0.6941, 0.6627,  ..., 0.7333, 0.8039, 0.7882],
          [0.7098, 0.6784, 0.6863,  ..., 0.7412, 0.7804, 0.7412],
          [0.7176, 0.6784, 0.7020,  ..., 0.7412, 0.7569, 0.6941]],

         [[0.5922, 0.5843, 0.6157,  ..., 0.5529, 0.5451, 0.6000],
          [0.5843, 0.6000, 0.5608,  ..., 0.5608, 0.5451, 0.5922],
          [0.5686, 0.6157, 0.5922,  ..., 0.5765, 0.5451, 0.5843],
          ...,
          [0.5294, 0.5059, 0.4510,  ..., 0.6157, 0.6784, 0.6627],
          [0.5529, 0.5137, 0.4824,  ..., 0.6078, 0.6627, 0.6235],
          [0.5922, 0.5216, 0.5137,  ..., 0.6157, 0.6392, 0.5765]],

         [[0.6392, 0.6235, 0.5922,  ..., 0.6549, 0.6157, 0.6471],
          [0.6392, 0.6471, 0.6000,  ..., 0.6392, 0.6157, 0.6314],
          [0.6314, 0.6706, 0.6471,  ..., 0